### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

### Compile contract from file

In [1]:
import json
import web3
from datetime import datetime
from web3 import Web3
from solcx import compile_source
import solcx
#from web3.contract import ConciseContract

In [2]:
# Read in Model contract code
with open('./data/Model.sol', 'r') as file:
    contract_source_code = file.read()

In [3]:
solcx.install_solc('0.6.2')
# Compile & Store Compiled source code
compiled_sol = solcx.compile_source(contract_source_code,solc_version='0.6.2')

In [4]:
#solcx.install_solc('0.6.2')

In [5]:
#compiled_sol

In [6]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Model']

In [7]:
#contract_interface

In [8]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [9]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))

In [10]:
# Use local Ganache GUI on macOS
#w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

In [11]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [12]:
w3.eth.accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

In [13]:
w3.eth.accounts[0]

'0x23cA5505ca6D97f439C66AaA6b371216406CDE4F'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [14]:
# Create contract blueprint
Contract = w3.eth.contract(abi=abi, bytecode=bytecode)

In [15]:
# Submit the transaction that deploys the contract
tx_hash = Contract.constructor().transact({"from": w3.eth.accounts[0]})

In [16]:
tx_hash

HexBytes('0x4a2ae0e5f01033ff8190048b1bab6fb40d5a3704414afa4d6497c7b557a14036')

### Obtain Transcation Receipt

In [17]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [18]:
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number",contract_block,".")

The contract is deployed with block number 69 .


In [19]:
# With obtain the final address of the contract 
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

The contract has the address 0x1B50F4E5b9a39c81805264AEB889e5d93389Df48 .


### Interact with contract

In [20]:
# Create python instance of deployed contract
Contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [21]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [22]:
accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

**Certification**  
```solidity

        function addCertifier(address newCertifier) public onlyOwner {}
        function deleteCertifier(address oldCertifier) public onlyOwner {}
        function checkIfCertified(address certified) public view returns(bool) {}
```

In [23]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
startTime = datetime.now()
#adding an event listener so we can check the eventss
event_filter = Contract.events['publishedModel'].create_filter(fromBlock='latest')


In [24]:
#checking if adding a new account to the certification works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before adding, certified = " + str(tx_hash))
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after adding, certified = " + str(tx_hash))

before adding, certified = False
after adding, certified = True


In [25]:
#checking if removing a certifier also works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before removing, certified = " + str(tx_hash))
tx_hash = Contract.functions.deleteCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after removing, certified = " + str(tx_hash))

before removing, certified = True
after removing, certified = False


In [26]:
print(w3.eth.get_block('latest'))

AttributeDict({'hash': HexBytes('0x772a8931c763efbb09a111279a6a16bbcb9dc45876f5e0686db72a80b6381a09'), 'parentHash': HexBytes('0xbb27d1a0760d8f3aa661830dd88ad71e4dd8eda9643db9428afa485b93562fb9'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'miner': '0x0000000000000000000000000000000000000000', 'stateRoot': HexBytes('0x618193a4650f053334f9a7475e545033ac9f3ae0c637c60c73e81e8c80444ac0'), 'transactionsRoot': HexBytes('0x7c3ca74933c28878b4b2b5372732a55d01377569f0bfb8fc74b6eee46992b323'), 'receiptsRoot': HexBytes('0xf4b08aeccec4c3ac186956b0303ad0f848a910c60f4c057efdd4efe7bc467e96'), 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [27]:
endTimeDataProvider = datetime.now()
elapsedTimeDataProvider = endTimeDataProvider - startTime
print("elapsedTimeDataProvider", elapsedTimeDataProvider)

elapsedTimeDataProvider 0:00:03.223675


In [28]:
#tx_hash

**Adding models**  
```solidity
        function publishModel(
                string memory _link,
                bytes memory _hash) public onlyOwner providerGasCost
```

**Updating Models**  
```solidity
        function updateHash(
                string memory _link,
                bytes memory newHash) public onlyOwner providerGasCost
```

**Certifying models**  
```solidity
        function certifyModel(bytes memory _hash) public certifier
```


**Getters and Setters for variables**  
```solidity
        function getVersion(bytes32 _hash)public view returns(uint256){}
        function getCertfication(uint256 _version) public view returns(bool){}
        function setLink(string memory _link) public onlyOwner(){}
        function getLink() public view returns(string memory){}
        function getModelCardHash() public view returns(bytes32){}


In [29]:
#Lets look at how the functions work whenever we are trying to upload a model,
#In here im using a test hash, this does not refer to a real hash, //you can add a real model here 

test_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777b93592d8d4d4a4096c57876a91c84f4a712')
#basically if model card hash is not 32 bytes needs to be changed
test_link = 'https/:www.somelinktomodel.com'
#please let me know if you have any troubles with this part, it might be that the model card is not a bytes32 hash, in that case i need to change the smart contract,
tx_hash = Contract.functions.publishModel(test_link, test_hash).transact({'from': accounts[0]})
#this model is not callable twice on the same contract, as it will recognise that the contract is published, keep that in mind while testing other functions in the same cell

#lets see if everything prints accordingly
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

https/:www.somelinktomodel.com, the link to the model
the model is certified: False


In [31]:
#lets see if we can print the event
def handle_event(event):
    print("Event:", event)

# Start event listener
for event in event_filter.get_all_entries():
    handle_event(event)

Event: AttributeDict({'args': AttributeDict({'sender': '0x23cA5505ca6D97f439C66AaA6b371216406CDE4F', 'link': 'https/:www.somelinktomodel.com', 'modelCardHash': b'0x05416460deb76d57af601be17e777b93592d8d4d4a4096c57876a91c84f4a712', 'version': 1}), 'event': 'publishedModel', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x1fb6d62ea13731007feaf8f9b5359ceef36219128dedb6205bc72c508a64aaa0'), 'address': '0x1B50F4E5b9a39c81805264AEB889e5d93389Df48', 'blockHash': HexBytes('0x26a09d34e5420556bfc5543480031b9e20e8c725e92e83db5b2e6d4e0c0c6c41'), 'blockNumber': 72})


As we can see, important properties are saved in the event, it is possible for anyone that has access to the blockchain to check the link, modelcardhash and the version of a certain model by checking the event. The event is for anyone to check. Other events include model certification and updating the model.

In [284]:
#let's see if we can certify the model, let's certify an account
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from': accounts[0]})
tx_hash = Contract.functions.certifyModel(test_hash).transact({'from': accounts[1]})
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

the model is certified: True


In [302]:
#Models can get certified, lets update a model
new_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777b93592d8d4d4sfdssc57876a91c84f4a712')
new_link = 'https/:www.somelinktonewmodel.com'
tx_hash = Contract.functions.updateHash(new_link, new_hash).transact({'from': accounts[0]})
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the " + str(m_version) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version).call()))
m_version_new = Contract.functions.getVersion(new_hash).call()
print("the " + str(m_version_new) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version_new).call()))
#making sure that the hash from the contract is the same as the input
assert str(Contract.functions.getMostRecentModelCardHash().call()) == str(new_hash)


https/:www.somelinktonewmodel.com, the link to the model
the 1th version of model is certified: True
the 15th version of model is certified: False


In [251]:
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)

elapsedTime1 0:12:59.385628


In [ ]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address8, False, False, False, False, True).estimateGas())
#print("Gas", Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).estimateGas())
#print("Gas", Contract.functions.giveResearchPurpose(test_address13, True, True, False, False, False).estimateGas())
#print("Gas", Contract.functions.givePerson(test_address13, True, False, True, False).estimateGas())
#print("Gas", Contract.functions.giveProfit(test_address13, True, False).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address13).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address14).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address15).estimateGas())